# Employer Review Prediction
## Reynara Ezra Pratama

## Background

## Business Understanding

## Data Understanding

1. `ReviewTitle` : Topik dari *review*.
2. `CompleteReview` : *Review* yang diberikan pegawai perusahaan.
3. `URL` : *Uniform Resource Locator*.
4. `Rating` : Penilaian yang diberikan pegawai perusahaan.
5. `ReviewDetails` : Detail mengenai *review*.

## Import Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import nltk

import warnings 
warnings.filterwarnings('ignore')

## Loading Dataset

**Load Data From Github**

In [ ]:
# url = "https://raw.githubusercontent.com/ReynaraEzra/Employer-Review/main/data_input/results.json"
# df = pd.read_json(url)

**Load Data From Local File**

In [ ]:
df = pd.read_json('data_input/result.json')